In [6]:
from python_speech_features import mfcc,logfbank
# from python_speech_features import logfbank
import scipy.io.wavfile as wav
import os
import glob
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier

mfcc_list = []
fbank_list = []
label_list = []
# i=0
data_audio = pd.read_csv('../Handout/train_label.csv')
def absoluteFilePaths(directory):
    for dirpath,_,filenames in os.walk(directory):
        for f in filenames:
            yield os.path.abspath(os.path.join(dirpath, f))
            
files = list(absoluteFilePaths("../Handout/Train/"))            
## reading each file ina loop and extracting features, taking average across rows and appending to the list
for filename in files:
    # print(filename)
    (rate, sig) = wav.read(filename)
    name = filename.split('/')[-1] 
    mfcc_feat = mfcc(sig, rate)
    fbank_feat = logfbank(sig, rate)
    mfcc_list.append(np.mean(mfcc_feat, axis=0))
    fbank_list.append(np.mean(fbank_feat, axis=0))
    label_list.append(data_audio[data_audio["File"] == name]["Label"].item())

## Merging all features and labels into a data frame
mfcc_names,fbank_names = [],[]
for i in range(len(mfcc_list[1])):
    mfcc_names.append('mfcc_'+str(i))
for i in range(len(fbank_list[1])):
    fbank_names.append('fbank_'+str(i))

data = pd.concat([pd.DataFrame(mfcc_list,columns=mfcc_names),pd.DataFrame(fbank_list,columns=fbank_names),
                  pd.DataFrame(label_list,columns=['label'])],axis=1)

## to see the number of records for each emotion
data.label.value_counts()
##to see top five rows
data.head()
##to see summary of data
data.describe()

## splitting data
train,test = train_test_split(data,test_size=0.15,stratify=data.label,random_state=12345)

## creating X and Y for training and testing
Y_train = train.pop('label')
X_train = train

Y_test = test.pop('label')
X_test = test

###Training the model

C = 1.0 # SVM regularization parameter
model = svm.SVC(kernel='rbf', C=1,gamma='auto').fit(X_train, Y_train)
#model = RandomForestClassifier(n_estimators=50).fit(X_train, Y_train)


###Evaluating the model
y_pred = model.predict(X_test)
conf_mat = confusion_matrix(Y_test, y_pred)
print(conf_mat)
print(classification_report(Y_test, y_pred))
print(accuracy_score(Y_test,y_pred))


/home/bigbrain/emotion-recognize/emr/lib/python3.6/site-packages/ipykernel_launcher.py:33: FutureWarning: `item` has been deprecated and will be removed in a future version


[[45 16 23  8  8 15]
 [20 43  4 36 10 21]
 [22  6 77  2 12 15]
 [ 4 28  0 97  1  4]
 [11 33 27 25 31  7]
 [19 30 23 19 11 32]]
              precision    recall  f1-score   support

           0       0.37      0.39      0.38       115
           1       0.28      0.32      0.30       134
           2       0.50      0.57      0.53       134
           3       0.52      0.72      0.60       134
           4       0.42      0.23      0.30       134
           5       0.34      0.24      0.28       134

    accuracy                           0.41       785
   macro avg       0.41      0.41      0.40       785
weighted avg       0.41      0.41      0.40       785

0.4140127388535032
